# 📡 06 - Model Inference to Alerts

This notebook uses a trained anomaly detection model to infer risk scores from sensor feature data, 
filters high-risk records (anomaly = 1), and writes alerts into a timestamped Delta table.

**Steps:**
- Read predictions from the latest model
- Filter only anomalous records
- Write them to a time-versioned table for downstream alerting or monitoring


In [0]:
from pyspark.sql.functions import col, current_timestamp, lit
from datetime import datetime

# Read predictions table
predictions = spark.read.table("arao.aerodemo.anomaly_predictions")

# Filter for high-risk predictions (anomaly = 1)
high_risk = predictions.filter(col("predicted_anomaly") == 1)

# Add audit metadata (timestamp and batch ID)
timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
high_risk_with_meta = (high_risk
    .withColumn("alert_generated_at", current_timestamp())
    .withColumn("batch_id", lit(timestamp_str)))

# Write to Delta table with overwrite mode (adjust if appending is needed)
high_risk_with_meta.write.format("delta").mode("overwrite").saveAsTable("arao.aerodemo.anomaly_alerts")

print(f"✅ Alerts written to 'arao.aerodemo.anomaly_alerts' with batch ID {timestamp_str}")

### 📌 Notes:
- `alert_generated_at`: records the timestamp of alert generation
- `batch_id`: uniquely identifies the batch for traceability or rollback
- This table can be used for integration with dashboards, alerting systems, or compliance reporting
